In [12]:
# Import libraries
import requests
import google.generativeai as genai
from dotenv import load_dotenv
import os
import pandas as pd
import tabulate
import googlemaps
import itertools
import numpy as np
from scipy.spatial import distance_matrix
from datetime import datetime

# Load environment variables
load_dotenv(override=True)
google_api_key = os.getenv("GOOGLE_API_KEY")
genai_api_key = os.getenv("GENAI_API_KEY")

# Set up Google Maps client
gmaps = googlemaps.Client(key=google_api_key)

# Set up Generative AI model
genai.configure(api_key=genai_api_key)
model = genai.GenerativeModel("gemini-1.5-flash")

In [1]:
def main():
    
    while True:
        start_location = input("Enter start location (or 'q' to quit): ").strip()
        if start_location.lower() == 'q':
            return
            
        end_location = input("Enter end location (or 'q' to quit): ").strip()
        if end_location.lower() == 'q':
            return
            
        if not start_location or not end_location:
            print("\nBoth start and end locations are required.")
            continue
            
        # Validate locations using trip_length_recommendation function
        try:
            recommended_days = trip_length_recommendation(start_location, end_location)
            print(f"\nRecommended trip duration: {recommended_days[0]}-{recommended_days[-1]} days")
        except Exception as e:
            print(f"\nError: Could not validate locations. Please try again.")
            print("Make sure to enter valid city names (e.g., 'City, State')")

        chosen_days = None
        while chosen_days is None:
            try:
                chosen_days = int(input(f"Choose the number of days for your trip ({recommended_days[0]}-{recommended_days[-1]}): ").strip())
                if chosen_days not in recommended_days:
                    print(f"Please choose a number within the recommended range ({recommended_days[0]}-{recommended_days[-1]}).")
                    chosen_days = None
            except ValueError:
                print("Invalid input. Please enter a number.")

        print(f"\nYou chose a trip duration of {chosen_days} days.")

        # Get user preferences
        interests = input("Enter your interests (e.g., 'nature, history, food'): ").strip().split(", ")
        wanted_stops = input("Enter the stops that must be in the trip: ").strip()

        stops = stops_recommendation(start_location, end_location, interests, wanted_stops, chosen_days)

        route_optimization(start_location, end_location, stops)
        break

main()

In [2]:
def trip_length_recommendation(start, end):
    if start and end:
        prompt = f"Only answer with a range of numbers (e.g. 1-2, 4-6). What is the ideal number of days for a relaxing road trip from {start} to {end}, including leisure stops along the way for sightseeing and unwinding?"

        ai_response = model.generate_content(prompt)
        range_str = ai_response.text
        
        start, end = map(int, range_str.split('-'))

        return list(range(start, end + 1))

In [3]:
def stops_recommendation(start, end, interests, wanted_stops, trip_days):

    prompt = f"Help me pick destinations that I should spend time at during my {trip_days} long road trip from {start} to {end}. I am interested in {', '.join(interests)}. I want to make {wanted_stops} stops along the way. Please provide a list of recommended stops that align with these interests, ensuring they are spaced out appropriately for the trip duration. Join any stops that are withing 30 miles of each other and do not include the end point. List the stops concisely, separated by semicolons, with no additional descriptions."

    ai_response = model.generate_content(prompt)
    stops = [stop.strip() for stop in ai_response.text.split(";") if stop.strip()]

    print(f'These are the stops suggested by AI:')
    for i, stop in enumerate(stops, 1):
        print(f'{i}. {stop}')
    
    if not stops:
        return "No stops suggested by the AI."
        
    # Ask user if they want to modify stops
    while True:
        modify = input("\nWould you like to modify the stops? (yes/no): ").lower()
        if modify == 'yes':
            action = input("Would you like to add or remove stops? (add/remove): ").lower()
            
            if action == 'add':
                add = input("Manually enter stops to add or ask AI fro suggestions (manual/ai): ").lower()
                if add == 'manual':
                    new_stop = input("Enter the name of the stop to add: ")

                elif add == "ai":
                    prompt = f"Give me more road trip destinations that I should go to considering my interests: {', '.join(interests)}. The trip is from {start} to {end} and is {trip_days} long. Do not include stops that you have already suggested: {', '.join(stops)}. List the stops concisely, separated by semicolons, with no additional descriptions." 
                    ai_response = model.generate_content(prompt)
                    new_stop = [stop.strip() for stop in ai_response.text.split(";") if stop.strip()]
                    print("\nAI suggested these additional stops:")
                    for i, stop in enumerate(new_stop, 1):
                        print(f"{i}. {stop}")

                while True:
                    try:
                        choices = input("\nEnter the numbers of stops you want to add separated by comma(e.g., 1,3,4): ")
                        indices = [int(x.strip()) - 1 for x in choices.split(',')]
                        if all(0 <= i < len(new_stop) for i in indices):
                            selected_stops = [new_stop[i] for i in indices]
                            stops.extend(selected_stops)
                            break
                        else:
                            print("Invalid number(s). Please try again.")

                    except ValueError:
                        print("Please enter valid numbers separated by commas.")
            elif action == 'remove':
                remove_num = int(input("Enter the number of the stop to remove (1-{}): "))
                if 1 <= remove_num <= len(stops):
                    removed_stop = stops.pop(remove_num - 1)
                    print(f"Removed: {removed_stop}")
                else:
                    print("Invalid stop number")
            
            print("\nUpdated stops:")
            for i, stop in enumerate(stops, 1):
                 print(f"{i}. {stop}")
        elif modify == 'no':
            break
        else:
            print("Please enter 'yes' or 'no'")

    return stops

In [21]:
def route_optimization(start, end, stops):
    try:
        # Get coordinates for the start, end, and stops
        start_coords = get_coordinates(start)
        end_coords = get_coordinates(end)
        stop_coords = [get_coordinates(stop) for stop in stops]

        if not start_coords or not end_coords or any(coord is None for coord in stop_coords):
            print("Error: Could not get coordinates for one or more locations.")
            return

        # Routes API endpoint
        routes_url = "https://routes.googleapis.com/directions/v2:computeRoutes"
        
        # Construct payload with coordinates
        payload = {
            "origin": {"location": {"latLng": start_coords}},
            "destination": {"location": {"latLng": end_coords}},
            "intermediates": [{"location": {"latLng": coords}} for coords in stop_coords],
            "travelMode": "DRIVE",
            "routingPreference": "TRAFFIC_UNAWARE"
        }
        
        field_mask = "routes.legs.distanceMeters,routes.legs.duration,routes.legs.steps.distanceMeters,routes.legs.steps.duration,routes.polyline.encodedPolyline"

        # Headers
        headers = {
            "Content-Type": "application/json",
            "X-Goog-Api-Key": google_api_key,
            "X-Goog-FieldMask": field_mask
        }

        # Send request
        response = requests.post(routes_url, headers=headers, json=payload)
        if response.status_code == 200:
            directions_result = response.json()
            optimized_route = directions_result['routes'][0]
            waypoint_order = optimized_route.get('waypointOrder', [])
            
            print(f"Original Waypoints: {stops}")
            print(f"Optimized Order of Waypoints: {[stops[i] for i in waypoint_order]}")
            
            total_distance = optimized_route['distanceMeters'] / 1000
            total_duration = optimized_route['duration'] / 60
            print(f"Total Optimized Distance: {total_distance:.2f} km")
            print(f"Total Optimized Duration: {total_duration:.2f} minutes")
            
            for i, leg in enumerate(optimized_route['legs']):
                print(f"\nLeg {i+1}: Start: {leg['startLocation']}, End: {leg['endLocation']}")
        else:
            print(f"Error: {response.status_code}, {response.text}")
    except Exception as e:
        print(f"Error: {e}")

In [20]:
def get_coordinates(address):
    geocode_result = gmaps.geocode(address)
    if geocode_result:
        # Extract latitude and longitude from the response
        location = geocode_result[0]['geometry']['location']
        return {"latitude": location["lat"], "longitude": location["lng"]}
    else:
        print(f"Error: No geocoding result for address: {address}")
        return None

In [38]:
import googlemaps

def get_route_polyline(optimized_route):
    # Mock function to extract lat/lng coordinates for testing
    return [{"lat": step['end_location']['lat'], "lng": step['end_location']['lng']} 
            for leg in optimized_route['legs'] for step in leg['steps']]

def format_time(seconds):
    """Convert seconds to hours and minutes format."""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    return f"{hours}h {minutes}m"

import googlemaps

# def verify_stops(stops):
#     gmaps = googlemaps.Client(key=google_api_key)
    
#     for stop in stops:
#         geocode_result = gmaps.geocode(stop)
        
#         if not geocode_result:
#             print(f"Stop '{stop}' is invalid: No result found.")
#         else:
#             # Print out the formatted address to verify it's the correct location
#             print(f"Stop '{stop}' is valid: {geocode_result[0]['formatted_address']}")

# # Example list of stops
# stop_locations = ["Nashville, TN", "Mammoth Cave National Park, KY", "Gaitlinburg, TN", "Chattanooga, TN", "Okefenokee Swamp, GA", "St. Marks National Wildlife Refuge, FL"]

# verify_stops(stop_locations)


# def route_optimization(start, end, stops):
#     try:
#         if not stops:
#             print("No stops to optimize")
#             return None

#         # Initialize Google Maps client
#         gmaps = googlemaps.Client(key=google_api_key)

#         # Get optimized directions
#         directions_result = gmaps.directions(
#             origin=start,
#             destination=end,
#             waypoints=stops,
#             optimize_waypoints=True,
#             mode="driving"
#         )

#         if not directions_result:
#             print("No directions found")
#             return None

#         optimized_route = directions_result[0]
        
#         # Display route summary
#         waypoint_order = optimized_route.get('waypoint_order', [])
#         ordered_stops = [stops[i] for i in waypoint_order]
        
#         print("**Original Stops:**", ", ".join(stops))
#         print("**Optimized Order:**", ", ".join(ordered_stops))

#         # Create route data
#         route_data = []
#         total_distance_miles = 0
#         total_duration_seconds = 0
        
#         for i, leg in enumerate(optimized_route['legs']):
#             dist_miles = leg['distance']['value'] * 0.000621371  # Convert meters to miles
#             dur_seconds = leg['duration']['value']  # Duration in seconds
#             total_distance_miles += dist_miles
#             total_duration_seconds += dur_seconds
            
#             route_data.append({
#                 'Segment': i + 1,
#                 'Start': leg['start_address'],
#                 'End': leg['end_address'],
#                 'Distance (miles)': round(dist_miles, 2),
#                 'Duration': format_time(dur_seconds)
#             })

#         # Display route metrics
#         print(f"Total Distance: {total_distance_miles:.2f} miles")
#         print(f"Total Duration: {format_time(total_duration_seconds)}")

#         # Show route visualization data (mocked map data for testing)
#         df_map = get_route_polyline(optimized_route)
#         print("Route Visualization Data:", df_map)

#         return directions_result

#     except googlemaps.exceptions.ApiError as e:
#         print(f"Google Maps API Error: {str(e)}")
#     except Exception as e:
#         print(f"Unexpected error: {str(e)}")
#     return None

# # Sample call to test the function
# start_location = "West Lafayette, IN"
# end_location = "Gainesville, FL"
# stop_locations = ["Nashville, TN", "Mammoth Cave National Park, KY", "Gaitlinburg, NC", "Chattanooga, TN", "Okefenokee Swamp, GA", "St. Marks National Wildlife Refuge, FL"]

# result = route_optimization(start_location, end_location, stop_locations)

import googlemaps

# Function to verify stops before route optimization
def verify_stops(stops):
    gmaps = googlemaps.Client(key=google_api_key)  # Replace with your actual API key
    
    for stop in stops:
        geocode_result = gmaps.geocode(stop)
        
        if not geocode_result:
            print(f"Stop '{stop}' is invalid: No result found.")
        else:
            print(f"Stop '{stop}' is valid: {geocode_result[0]['formatted_address']}")

# Function to perform route optimization
def route_optimization(start, end, stops):
    try:
        if not stops:
            print("No stops to optimize")
            return None

        # Initialize Google Maps client
        gmaps = googlemaps.Client(key=google_api_key)  # Replace with your actual API key

        # Get optimized directions
        directions_result = gmaps.directions(
            origin=start,
            destination=end,
            waypoints=stops,
            optimize_waypoints=True,
            mode="driving"
        )

        if not directions_result:
            print("No directions found")
            return None

        optimized_route = directions_result[0]
        
        # Display route summary
        waypoint_order = optimized_route.get('waypoint_order', [])
        ordered_stops = [stops[i] for i in waypoint_order]
        
        print("**Original Stops:**", ", ".join(stops))
        print("**Optimized Order:**", ", ".join(ordered_stops))

        # Create route data
        route_data = []
        total_distance_miles = 0
        total_duration_seconds = 0
        
        for i, leg in enumerate(optimized_route['legs']):
            dist_miles = leg['distance']['value'] * 0.000621371  # Convert meters to miles
            dur_seconds = leg['duration']['value']  # Duration in seconds
            total_distance_miles += dist_miles
            total_duration_seconds += dur_seconds
            
            route_data.append({
                'Segment': i + 1,
                'Start': leg['start_address'],
                'End': leg['end_address'],
                'Distance (miles)': round(dist_miles, 2),
                'Duration': format_time(dur_seconds)
            })

        # Display route metrics
        print(f"Total Distance: {total_distance_miles:.2f} miles")
        print(f"Total Duration: {format_time(total_duration_seconds)}")

        # Show route visualization data (mocked map data for testing)
        df_map = get_route_polyline(optimized_route)
        print("Route Visualization Data:", df_map)

        return directions_result

    except googlemaps.exceptions.ApiError as e:
        print(f"Google Maps API Error: {str(e)}")
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
    return None

# Sample call to test the function
start_location = "West Lafayette, IN"
end_location = "Gainesville, FL"
stop_locations = ["Nashville, TN", "Mammoth Cave National Park, KY", "Tallahasee, FL", "Gatlinburg, TN", "Chatanooga, TN", "Great Smoky Mountains National Park, TN"]
verify_stops(stop_locations)

result = route_optimization(start_location, end_location, stop_locations)


Stop 'Nashville, TN' is valid: Nashville, TN, USA
Stop 'Mammoth Cave National Park, KY' is valid: Mammoth Cave, KY 42259, USA
Stop 'Tallahasee, FL' is valid: Tallahassee, FL, USA
Stop 'Gatlinburg, TN' is valid: Gatlinburg, TN 37738, USA
Stop 'Chatanooga, TN' is valid: Chattanooga, TN, USA
Stop 'Great Smoky Mountains National Park, TN' is valid: Great Smoky Mountains National Park, United States
**Original Stops:** Nashville, TN, Mammoth Cave National Park, KY, Tallahasee, FL, Gatlinburg, TN, Chatanooga, TN, Great Smoky Mountains National Park, TN
**Optimized Order:** Mammoth Cave National Park, KY, Nashville, TN, Great Smoky Mountains National Park, TN, Gatlinburg, TN, Chatanooga, TN, Tallahasee, FL
Total Distance: 1274.16 miles
Total Duration: 20h 49m
Route Visualization Data: [{'lat': 40.4257597, 'lng': -86.9080275}, {'lat': 40.425733, 'lng': -86.9069293}, {'lat': 40.4266521, 'lng': -86.90693759999999}, {'lat': 40.4252383, 'lng': -86.8994151}, {'lat': 40.42468239999999, 'lng': -86.89